# Cleaning Messy Data

We will be working with the Bertelsmann Transformation Index from 2020. It's messy. Let's get into it. The data is from [this site](https://www.bti-project.org/en/meta/downloads.html). If you look at it in Excel, it makes a good bit of sense, but trying to interact with it in an automated manner doesn't work as well. 

## Reading in and Initial Cleaning

In [1]:
import pandas as pd
from pathlib import Path

file_path = Path.cwd() / 'data' / 'bti.xlsx'
df = pd.read_excel(file_path)

df.head()

,Regions:\n1 | East-Central and Southeast Europe\n2 | Latin America and the Caribbean\n3 | West and Central Africa\n4 | Middle East and North Africa\n5 | Southern and Eastern Africa\n6 | Post-Soviet Eurasia\n7 | Asia and Oceania,Region,Ranking Status Index,S | Status Index,Ranking Democracy Status,SI | Democracy Status,Q1 | Stateness,Q1.1 | Monopoly on the use of force,Q1.2 | State identity,Q1.3 | No interference of religious dogmas,...,.2,G | Governance Index.1,Category.3,.3,Q13 | Level of Difficulty.1,Category.4,.4,GII | Governance Performance.1,Category.5,.5
0,Afghanistan,7,123,3.213095,118,3.283333,4.00,2,7,3,...,Very limited,4.36,3,Moderate,8.9,1,Massive,4.466667,4,Flawed
1,Albania,1,28,6.842857,33,7.150000,9.00,10,9,10,...,Limited,5.95,2,Good,4.5,3,Moderate,6.783333,2,Sound
2,Algeria,4,83,5.028571,80,4.700000,7.50,8,8,6,...,Limited,4.42,3,Moderate,5.4,3,Moderate,4.933333,4,Flawed
3,Angola,5,105,4.164286,84,4.650000,7.25,8,7,10,...,Very limited,4.23,4,Weak,5.9,3,Moderate,4.650000,4,Flawed
4,Argentina,2,19,7.289286,16,8.150000,9.00,8,10,10,...,Limited,6.10,2,Good,2.8,4,Minor,7.266667,2,Sound


There are some obvious issues here with the columns. The BTI is made up of a number of sections and subquestions and the columns reflect that, but for our purposes that's immaterial. Let's get rid of the stuff before the pipe in the columns. Along the way I'm also getting rid of the whitespace before and after some of the column names.  

In [2]:
df.columns = [c.split('|')[1].strip() if '|' in c else c.strip() for c in df.columns]
print(df.columns.tolist())


['East-Central and Southeast Europe\n2', 'Region', 'Ranking Status Index', 'Status Index', 'Ranking Democracy Status', 'Democracy Status', 'Stateness', 'Monopoly on the use of force', 'State identity', 'No interference of religious dogmas', 'Basic administration', 'Political Participation', 'Free and fair elections', 'Effective power to govern', 'Association / assembly rights', 'Freedom of expression', 'Rule of Law', 'Separation of powers', 'Independent judiciary', 'Prosecution of office abuse', 'Civil rights', 'Stability of Democratic Institutions', 'Performance of democratic institutions', 'Commitment to democratic institutions', 'Political and Social Integration', 'Party system', 'Interest groups', 'Approval of democracy', 'Social capital', 'Ranking Economy Status', 'Economy Status', 'Level of Socioeconomic Development', 'Socioeconomic barriers', 'Organization of the Market and Competition', 'Market organization', 'Competition policy', 'Liberalization of foreign trade', 'Banking sys

We can see that there are also some unnamed columns and trailing crud still in there. Let's try creating a function to take care of this. You also could do the same thing with a dictionary and the rename function in Pandas. 

I'm also going to drop some of the last columns because I don't need them. 

In [3]:
df = df.drop(['Total', 'Trend', 'Categories', 'Political System', 'Trend Governance', 'Region'], axis = 1)
df = df.drop(df.columns[df.columns.str.contains('^Unnamed', case = False)], axis = 1)
df = df.drop(df.columns[df.columns.str.contains('^Category', case = False)], axis = 1)



In [4]:
def fix_bti_cols(col):
    if col.endswith('\n2'):
        return 'Country'
    if col == '':
        return 'Status Index Description'
    if col == '.1':
        return 'Democracy Status Description'
    if col == '.2':
        return 'Economy Status Description'
    if col == '.3':
        return 'Governance Index Description'
    if col == '.4':
        return 'Level of Difficulty Description'
    if col == '.5':
        return 'Governance Performance Description'
    if col.endswith('.1'):
        return col.strip('.1')
    return col
    
df.columns = [fix_bti_cols(c) for c in df.columns.tolist()]
print(df.columns.tolist())

['Country', 'Ranking Status Index', 'Status Index', 'Ranking Democracy Status', 'Democracy Status', 'Stateness', 'Monopoly on the use of force', 'State identity', 'No interference of religious dogmas', 'Basic administration', 'Political Participation', 'Free and fair elections', 'Effective power to govern', 'Association / assembly rights', 'Freedom of expression', 'Rule of Law', 'Separation of powers', 'Independent judiciary', 'Prosecution of office abuse', 'Civil rights', 'Stability of Democratic Institutions', 'Performance of democratic institutions', 'Commitment to democratic institutions', 'Political and Social Integration', 'Party system', 'Interest groups', 'Approval of democracy', 'Social capital', 'Ranking Economy Status', 'Economy Status', 'Level of Socioeconomic Development', 'Socioeconomic barriers', 'Organization of the Market and Competition', 'Market organization', 'Competition policy', 'Liberalization of foreign trade', 'Banking system', 'Monetary and Fiscal Stability', 

Ok, now the columns look decent. Let's make the index be the current Country column.

In [5]:
df = df.set_index('Country')

<bound method NDFrame.head of              Ranking Status Index  Status Index  Ranking Democracy Status  \
Country                                                                     
Afghanistan                   123      3.213095                       118   
Albania                        28      6.842857                        33   
Algeria                        83      5.028571                        80   
Angola                        105      4.164286                        84   
Argentina                      19      7.289286                        16   
...                           ...           ...                       ...   
Venezuela                     128      2.773810                       120   
Vietnam                        90      4.819048                       107   
Yemen                         136      1.517857                       136   
Zambia                         76      5.160714                        68   
Zimbabwe                      115      3.68333

In [6]:
df.head()

,Ranking Status Index,Status Index,Ranking Democracy Status,Democracy Status,Stateness,Monopoly on the use of force,State identity,No interference of religious dogmas,Basic administration,Political Participation,...,Democracy Status,Democracy Status Description,Economy Status,Economy Status Description,Governance Index,Governance Index Description,Level of Difficulty,Level of Difficulty Description,Governance Performance,Governance Performance Description
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,123,3.213095,118,3.283333,4.00,2,7,3,4,3.75,...,3.28,Hard-line autocracy,3.14,Very limited,4.36,Moderate,8.9,Massive,4.466667,Flawed
Albania,28,6.842857,33,7.150000,9.00,10,9,10,7,7.75,...,7.15,Defective democracy,6.54,Limited,5.95,Good,4.5,Moderate,6.783333,Sound
Algeria,83,5.028571,80,4.700000,7.50,8,8,6,8,4.25,...,4.70,Moderate autocracy,5.36,Limited,4.42,Moderate,5.4,Moderate,4.933333,Flawed
Angola,105,4.164286,84,4.650000,7.25,8,7,10,4,4.25,...,4.65,Moderate autocracy,3.68,Very limited,4.23,Weak,5.9,Moderate,4.650000,Flawed
Argentina,19,7.289286,16,8.150000,9.00,8,10,10,8,9.00,...,8.15,Democracy in consolidation,6.43,Limited,6.10,Good,2.8,Minor,7.266667,Sound


In [8]:
print(df.loc['Afghanistan'].tolist())

[123, 3.2130952380952382, 118, 3.2833333333333337, 4.0, 2, 7, 3, 4, 3.75, 4, 2, 5, 4, 3.0, 4, 3, 3, 2, 3.0, 3, 3, 2.6666666666666665, 3, 3, nan, 2, 122, 3.142857142857143, 1, 1, 4.0, 4, 4, 4, 4, 4.5, 5, 4, 3.5, 2, 5, 2.5, 2, 3, 4, 4, 2.5, 3, 2, 89, 4.359135802469137, 8.916666666666666, 10, 10, 9, 9, 8, 7.5, 4.466666666666667, 4.666666666666667, 5, 4, 5, 3.0, 3, 3, 3, 4.2, 3, 4, 5, 4, 5.0, 6.0, 6, 5, 7, 3.0166666666666666, 3.2833333333333337, 0.26666666666666705, 2.892857142857143, 3.142857142857143, 0.25, 4.023279320987655, 4.359135802469137, 4, 2, 5, 4, 4, 2, 3.0, 'Aut.', 3.21, 'Failed', 3.28, 'Hard-line autocracy', 3.14, 'Very limited', 4.36, 'Moderate', 8.9, 'Massive', 4.466666666666667, 'Flawed']


No NaNs! So our work is done. 